<a href="https://colab.research.google.com/github/jsvon/NLP-projects-/blob/main/Token_classification_huggingface_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfsn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package git-lfsn


In [2]:
!git config --global user.email "jumavonne10@gmail.com"
!git config --global user.name "Juma Vonne"

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
pip install datasets==3.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [5]:
from datasets import load_dataset

raw_datasets = load_dataset("conll2003")
trust_remote_code=True

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

conll2003.py: 0.00B [00:00, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [6]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [7]:
raw_datasets["train"][139]["tokens"]

['Mujahideen',
 'Khalq',
 'said',
 'Iranian',
 'troops',
 'had',
 'also',
 'been',
 'shelling',
 'KDP',
 'positions',
 'in',
 'Qasri',
 'region',
 'in',
 'Suleimaniya',
 'province',
 'near',
 'the',
 'Iranian',
 'border',
 'over',
 'the',
 'last',
 'two',
 'days',
 '.']

In [8]:
raw_datasets["train"][139]["ner_tags"]

[3,
 4,
 0,
 7,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 5,
 0,
 0,
 5,
 0,
 0,
 0,
 7,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [9]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [10]:
from datasets import Sequence, ClassLabel

print(Sequence)
print(ClassLabel)

<class 'datasets.features.features.Sequence'>
<class 'datasets.features.features.ClassLabel'>


In [11]:
Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], names_file=None, id=None), length=-1, id=None)

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [12]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [13]:
#decoding the labels
words = raw_datasets["train"][139]["tokens"]
labels = raw_datasets["train"][139]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
  full_label = label_names[label]
  max_length = max(len(word), len(full_label))
  line1 += word + " " * (max_length - len(word) + 1)
  line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

Mujahideen Khalq said Iranian troops had also been shelling KDP   positions in Qasri region in Suleimaniya province near the Iranian border over the last two days . 
B-ORG      I-ORG O    B-MISC  O      O   O    O    O        B-ORG O         O  B-LOC O      O  B-LOC       O        O    O   B-MISC  O      O    O   O    O   O    O 


**Creating a tokenizer object**

In [14]:
#Creating a tokenizer object

from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [15]:
#Checking if its a fast tokenizer
tokenizer.is_fast

True

In [16]:
#To tokenize a pre-tokenized input, we can use our tokenizer as usual and just add is_split_into_words=True:
inputs = tokenizer(raw_datasets["train"][139]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'Mu',
 '##jah',
 '##ide',
 '##en',
 'K',
 '##hal',
 '##q',
 'said',
 'Iranian',
 'troops',
 'had',
 'also',
 'been',
 'shell',
 '##ing',
 'K',
 '##DP',
 'positions',
 'in',
 'Q',
 '##as',
 '##ri',
 'region',
 'in',
 'Sul',
 '##eim',
 '##ani',
 '##ya',
 'province',
 'near',
 'the',
 'Iranian',
 'border',
 'over',
 'the',
 'last',
 'two',
 'days',
 '.',
 '[SEP]']

In [17]:
#mapping each token to the corresponding word
inputs.word_ids()

[None,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 8,
 9,
 9,
 10,
 11,
 12,
 12,
 12,
 13,
 14,
 15,
 15,
 15,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 None]

In [18]:
#ALigning labels with tokens
def align_labels_with_tokens(labels, word_ids):
  new_labels = []
  current_word = None
  for word_id in word_ids:
    if word_id != current_word:
      #start of a new word
      current_word = word_id
      label = -100 if word_id is None else labels[word_id]
      new_labels.append(label)
    elif word_id is None:
      # Special token
      new_labels.append(-100)
    else:
      # Same word as previous token
      label = labels[word_id]
      # If the label is B-XXX we change it to I-XXX
      if label % 2 == 1:
        label += 1
      new_labels.append(label)

  return new_labels

In [19]:
# -100 is added to the 2 special tokens at beginning and end
# align_labels_with_tokens is applied on all labels
labels = raw_datasets["train"][139]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[3, 4, 0, 7, 0, 0, 0, 0, 0, 3, 0, 0, 5, 0, 0, 5, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0]
[-100, 3, 4, 4, 4, 4, 4, 4, 0, 7, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 5, 6, 6, 0, 0, 5, 6, 6, 6, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, -100]


In [20]:
# function that processes list of examples to take advantage of the fast tokenizer speed

def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(
      examples["tokens"], truncation=True, is_split_into_words=True
  )
  all_labels = examples["ner_tags"]
  new_labels = []
  for i, labels in enumerate(all_labels):
    word_ids = tokenized_inputs.word_ids(i)
    new_labels.append(align_labels_with_tokens(labels, word_ids))

  tokenized_inputs["labels"] = new_labels
  return tokenized_inputs

In [21]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

**Data collation**

In [22]:
# padding of labels should be same as the inputs
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)


In [23]:
#testing on a few samples
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

In [24]:
#compairing the padded tokens to labels for first and second element

for i in range(2):
  print(tokenized_datasets["train"][i]["labels"])
  print(tokenizer.decode(batch["input_ids"][i]))

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]
[CLS] EU rejects German call to boycott British lamb. [SEP]
[-100, 1, 2, -100]
[CLS] Peter Blackburn [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


**Metrics**

In [25]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=ef6001a59971a370e4253c3f24f8e91cf14acb6da4f65645d6792c9047e724c3
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [26]:
import evaluate
metric = evaluate.load("seqeval")

In [27]:
#example of how the metric works
labels = raw_datasets["train"][139]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-ORG',
 'I-ORG',
 'O',
 'B-MISC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'O',
 'O',
 'B-LOC',
 'O',
 'O',
 'B-LOC',
 'O',
 'O',
 'O',
 'B-MISC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [28]:
#Creating fake predictions at index 2 to see how the metric works
#Metric takes a list of predictions not just one and a list of labels
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'LOC': {'precision': np.float64(1.0),
  'recall': np.float64(1.0),
  'f1': np.float64(1.0),
  'number': np.int64(2)},
 'MISC': {'precision': np.float64(1.0),
  'recall': np.float64(1.0),
  'f1': np.float64(1.0),
  'number': np.int64(2)},
 'ORG': {'precision': np.float64(1.0),
  'recall': np.float64(1.0),
  'f1': np.float64(1.0),
  'number': np.int64(2)},
 'overall_precision': np.float64(1.0),
 'overall_recall': np.float64(1.0),
 'overall_f1': np.float64(1.0),
 'overall_accuracy': 1.0}

In [29]:
import numpy as np

def compute_metrics(eval_preds):
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)

  #Removing ignored index (Special tokens) and convert to labels
  true_labels = [[label_names[1] for l in label if l != -100] for label in labels]
  true_predictions = [
      [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(predictions, labels)
  ]
  all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
  return {
      "precision": all_metrics["overall_precision"],
      "recall": all_metrics["overall_recall"],
      "f1": all_metrics["overall_f1"],
      "accuracy": all_metrics["overall_accuracy"],
  }

**DEFINING THE MODEL**

In [30]:
# token classification problem uses AutoModelForTokenClassification
# always pass along some information on the number of labels we have
#dictionaries used are
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}


In [31]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
#Double checking that our model has the right number of labels
model.config.num_labels

9

**FINE-TUNING THE MODEL**

In [33]:
#defining the training arguements
from huggingface_hub import notebook_login

notebook_login()

In [34]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [35]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=tokenizer,
)
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jumavonne10 (jumavonne10-kabarak-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.075300,0.065090,0.040442,0.003665,0.006720,0.027550
2,0.034900,0.066922,0.037864,0.003370,0.006190,0.027330
3,0.022900,0.063048,0.038061,0.003385,0.006217,0.027286


/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=5268, training_loss=0.06676841298105506, metrics={'train_runtime': 670.1996, 'train_samples_per_second': 62.851, 'train_steps_per_second': 7.86, 'total_flos': 920771584279074.0, 'train_loss': 0.06676841298105506, 'epoch': 3.0})

In [36]:
trainer.push_to_hub(commit_message="Training complete")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...ned-ner/training_args.bin: 100%|##########| 5.84kB / 5.84kB            

  ...137206.ae6049f52f3c.220.0: 100%|##########| 9.24kB / 9.24kB            

  ...ned-ner/model.safetensors:   6%|5         | 25.0MB /  431MB            

CommitInfo(commit_url='https://huggingface.co/jvon65434/bert-finetuned-ner/commit/ad8ed2f7e8f6610ef25e46fc284199bd51ac33e1', commit_message='Training complete', commit_description='', oid='ad8ed2f7e8f6610ef25e46fc284199bd51ac33e1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jvon65434/bert-finetuned-ner', endpoint='https://huggingface.co', repo_type='model', repo_id='jvon65434/bert-finetuned-ner'), pr_revision=None, pr_num=None)

**CUSTOM TRAINING LOOP**

In [37]:
#training preparation
# 1. building DataLoaders from dataset
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

eval_dataloader = DataLoader(
    tokenized_datasets["validation"],
    collate_fn=data_collator,
    batch_size=8,
)

In [38]:
#Reinstatiating model to make sure its not finetuning from before but starting from Bert pretrained model
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
# Using AdamW optimizer
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [40]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [41]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

**TRAINING LOOP**

The training in itself, which is the classic iteration over the train_dataloader, forward pass through the model, then backward pass and optimizer step.
The evaluation, in which there is a novelty after getting the outputs of our model on a batch: since two processes may have padded the inputs and labels to different shapes, we need to use accelerator.pad_across_processes() to make the predictions and labels the same shape before calling the gather() method. If we don’t do this, the evaluation will either error out or hang forever. Then we send the results to metric.add_batch() and call metric.compute() once the evaluation loop is over.
Saving and uploading, where we first save the model and the tokenizer, then call repo.push_to_hub(). Notice that we use the argument blocking=False to tell the 🤗 Hub library to push in an asynchronous process. This way, training continues normally and this (long) instruction is executed in the background.

In [52]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "bert-finetuned-ner-accelerate"
repo_name = get_full_repo_name(model_name)
repo_name

'jvon65434/bert-finetuned-ner-accelerate'

In [54]:
output_dir = "bert-finetuned-ner-accelerate"
repo = Repository(output_dir, clone_from=repo_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/jvon65434/bert-finetuned-ner-accelerate into local empty directory.


In [55]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [56]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/5268 [00:00<?, ?it/s]

epoch 0: {'precision': np.float64(0.40298400913763566), 'recall': np.float64(0.08520754716981133), 'f1': np.float64(0.140671334944803), 'accuracy': 0.7745736191726629}
epoch 1: {'precision': np.float64(0.9436216762032985), 'recall': np.float64(0.922203947368421), 'f1': np.float64(0.9327898852104475), 'accuracy': 0.9856213575086831}
epoch 2: {'precision': np.float64(0.947997307303938), 'recall': np.float64(0.9258711374095989), 'f1': np.float64(0.9368035922168634), 'accuracy': 0.9861364572908695}


In [57]:
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

***USING THE FINE-TUNED MODEL***

In [61]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "huggingface-course/bert-finetuned-ner"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("My name is Vonne and I work with Jamii in Nairobi.")

Device set to use cuda:0


[{'entity_group': 'PER',
  'score': np.float32(0.9983101),
  'word': 'Vonne',
  'start': 11,
  'end': 16},
 {'entity_group': 'PER',
  'score': np.float32(0.99385476),
  'word': 'Jamii',
  'start': 33,
  'end': 38},
 {'entity_group': 'LOC',
  'score': np.float32(0.99914896),
  'word': 'Nairobi',
  'start': 42,
  'end': 49}]